# EDA Notebook

## 1. Importar librerías necesarias
Importar pandas, numpy, matplotlib y seaborn para el análisis y visualización de datos.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

Matplotlib is building the font cache; this may take a moment.


## 2. Cargar el archivo CSV
Solicitar al usuario la ruta del archivo .csv y cargarlo en un DataFrame de pandas.

In [11]:
# Change this path to point to your data file
file_path = "../data/raw/2064.csv"
df = pd.read_csv(file_path, sep=";", encoding="latin-1", na_values=[".", ""])

## 3. Visualizar las primeras filas del dataset
Mostrar las primeras filas del DataFrame usando head() para obtener una vista general de los datos.

In [30]:
df.head()

,ï»¿Provincias con mayor nÃºmero de pernoctaciones,Establecimientos y personal empleado (parcelas),Periodo,Total,year
0,02 Albacete,NÃºmero de establecimientos abiertos estimados,2025M12,NaN,2025
1,02 Albacete,NÃºmero de establecimientos abiertos estimados,2025M11,NaN,2025
2,02 Albacete,NÃºmero de establecimientos abiertos estimados,2025M10,6,2025
3,02 Albacete,NÃºmero de establecimientos abiertos estimados,2025M09,5,2025
4,02 Albacete,NÃºmero de establecimientos abiertos estimados,2025M08,7,2025


## 4. Explorar información general del dataset
Utilizar info() y shape para mostrar información sobre el número de filas, columnas y tipos de datos.

In [29]:
df.info()
print(f"Número de filas: {df.shape[0]}")
print(f"Número de columnas: {df.shape[1]}")

<class 'pandas.DataFrame'>
Index: 37800 entries, 0 to 88055
Data columns (total 5 columns):
 #   Column                                             Non-Null Count  Dtype
---  ------                                             --------------  -----
 0   ï»¿Provincias con mayor nÃºmero de pernoctaciones  37800 non-null  str  
 1   Establecimientos y personal empleado (parcelas)    37800 non-null  str  
 2   Periodo                                            37800 non-null  str  
 3   Total                                              24169 non-null  str  
 4   year                                               37800 non-null  int64
dtypes: int64(1), str(4)
memory usage: 1.7 MB
Número de filas: 37800
Número de columnas: 5


# (EXTRA 1). Limpieza primaria del dataset
Realizamos una primera limpieza exploratoria del dataset para tratar los datos a partir del 2017. Para ello, debemos de tratar el campo que almacena este dato como un entero

In [22]:
df["year"] = df["Periodo"].str[:4].astype(int)
df = df[df["year"] >= 2017]

Viendo los datos de los otros campos del notebook, vemos que quedan **37800** filas dentro de el CSV limpio, lo cual tiene sentido puesto que 50 × 7 × 12 × 9 ≈ 37.800 (50 provincias, 7 variables, 12 meses, 9 años)

## Convertir TOTAL en float sin romper los datos

Primeramente vamos a ver que datos tenemos para ver que limpieza poder aplicar

In [33]:
df["Total"].unique()[:20]

<StringArray>
[    nan,     '6',     '5',     '7',     '4',     '3',     '0',     '8',
     '9', '3.541', '2.878', '3.623', '3.313', '3.099', '2.915', '2.912',
 '2.611', '3.028', '3.109', '3.310']
Length: 20, dtype: str

Visto estos resultados, tenemos que realizar una transforamción a float pero primeramente tenemos que quitar los puntos de miles, para que no sean tratados como decimales. 

In [40]:
# Eliminar puntos de miles
df["Total"] = df["Total"].str.replace(".", "", regex=False)

# Convertir a numérico
df["Total"] = pd.to_numeric(df["Total"], errors="coerce")

# Verificar
print(df["Total"].dtype)
df["Total"].head(10)

float64


0    NaN
1    NaN
2    6.0
3    5.0
4    7.0
5    6.0
6    5.0
7    5.0
8    5.0
9    4.0
Name: Total, dtype: float64

In [ ]:
df["Total"].max()

np.float64(937197.0)

## Convertir PERIODO en fecha real


In [61]:
# Extraer mes
df["month"] = df["Periodo"].str[5:7].astype(int)

# Crear fecha
df["date"] = pd.to_datetime(dict(year=df["year"], month=df["month"], day=1))

df[["Periodo", "date"]].head()

,Periodo,date
0,2025M12,2025-12-01
1,2025M11,2025-11-01
2,2025M10,2025-10-01
3,2025M09,2025-09-01
4,2025M08,2025-08-01


## Pivotar variables
Actualmente las 7 métricas están en filas, pero necesitamos convertirlas en columnas
El objetivo sería obtener algo de este estilo: 
|Provincia|Fecha|Establecimientos|Parcelas|Ocupacion|Personal|

Para ello primeramente veamos como se llaman exactamente las variables

In [68]:
df["Establecimientos y personal empleado (parcelas)"].unique()

<StringArray>
[  'NÃºmero de establecimientos abiertos estimados',
                      'NÃºmero de plazas estimadas',
                              'NÃºmero de parcelas',
                                'Parcelas ocupadas',
                 'Grado de ocupaciÃ³n por parcelas',
 'Grado de ocupaciÃ³n de parcelas en fin de semana',
                                'Personal empleado']
Length: 7, dtype: str

Primeramente, vamos a renombrar las columnas con nombres largos a nombres más simples

In [69]:
df = df.rename(columns={
    "ï»¿Provincias con mayor nÃºmero de pernoctaciones": "provincia",
    "Establecimientos y personal empleado (parcelas)": "variable"
})

In [76]:
df.columns

Index(['provincia', 'variable', 'Periodo', 'Total', 'year', 'month', 'date'], dtype='str')

Y finalmente, lo que vamos a hacer es que
* Para cada provincia
* Para cada fecha
Cada variable pasará a ser una columna y su valor será total


In [83]:
df_pivot = df.pivot_table(
    index=["provincia", "date"],
    columns="variable",
    values="Total",
    aggfunc="first"
).reset_index()

df_pivot.head()

variable,provincia,date,Grado de ocupaciÃ³n de parcelas en fin de semana,Grado de ocupaciÃ³n por parcelas,NÃºmero de establecimientos abiertos estimados,NÃºmero de parcelas,NÃºmero de plazas estimadas,Parcelas ocupadas,Personal empleado
0,01 Araba/Ãlava,2024-03-01,NaN,NaN,3.0,500.0,1586.0,7876.0,34.0
1,01 Araba/Ãlava,2024-04-01,NaN,NaN,3.0,540.0,1712.0,8107.0,59.0
2,01 Araba/Ãlava,2024-05-01,NaN,NaN,3.0,531.0,1712.0,9050.0,65.0
3,01 Araba/Ãlava,2024-06-01,77.0,NaN,3.0,540.0,1712.0,9110.0,72.0
4,01 Araba/Ãlava,2024-07-01,NaN,NaN,3.0,531.0,1712.0,10040.0,72.0


Finalmente, lo que vamos a hacer es cambiar los nombres de las columnas para que sea más fácil trabajar con ellas:

In [91]:
df_pivot = df_pivot.rename(columns={
    "NÃºmero de establecimientos abiertos estimados": "Establecimientos",
    "NÃºmero de plazas estimadas": "Plazas",
    "NÃºmero de parcelas": "Parcelas",
    "Parcelas ocupadas": "Parcelas_Ocupadas",
    "Grado de ocupaciÃ³n por parcelas": "Ocupacion_Parcelas",
    "Grado de ocupaciÃ³n de parcelas en fin de semana": "Ocupacion_finde",
    "Personal empleado": "Personal"
})

## 5. Describir variables numéricas y categóricas
Usar describe() para variables numéricas y value_counts() para variables categóricas.

In [98]:
# Numerical variables
df.describe()

# Categorical values (Example for a columnn)
# df['category_column'].value_counts()

,Total,year,month,date
count,17372.000000,37800.000000,37800.000000,37800
mean,13849.915784,2021.000000,6.500000,2021-06-16 08:26:40
min,0.000000,2017.000000,1.000000,2017-01-01 00:00:00
25%,50.750000,2019.000000,3.750000,2019-03-24 06:00:00
50%,1551.500000,2021.000000,6.500000,2021-06-16 00:00:00
75%,8026.750000,2023.000000,9.250000,2023-09-08 12:00:00
max,937197.000000,2025.000000,12.000000,2025-12-01 00:00:00
std,45196.226102,2.582023,3.452098,NaN


## 6. Visualizar valores nulos y duplicados
Identificar y visualizar la cantidad de valores nulos y filas duplicadas en el dataset.

In [93]:
# Null values
df.isnull().sum()

# Duplicated rows
df.duplicated().sum()

np.int64(0)

## 7. Análisis univariado de variables numéricas
Realizar histogramas y boxplots para analizar la distribución de las variables numéricas.

In [94]:
# Histogram for a numerical variable
# df['numeric_column'].hist()
# plt.show()

# Boxplot
# df.boxplot(column='numeric_column')
# plt.show()

## 8. Análisis univariado de variables categóricas
Realizar gráficos de barras para analizar la frecuencia de las variables categóricas.

In [95]:
# Bar plot for a categorical variable
# df['categorical_column'].value_counts().plot(kind='bar')
# plt.show()

## 9. Análisis bivariado entre variables
Explorar relaciones entre variables numéricas y categóricas mediante scatterplots y tablas cruzadas.

In [96]:
# Scatterplot
# plt.scatter(df['numeric_column_x'], df['numeric_column_y'])
# plt.show()

# Crosstab for two categorical variables   
# pd.crosstab(df['categorical_column1'], df['categorical_column2'])

## 10. Visualización de correlaciones
Calcular y visualizar la matriz de correlación entre variables numéricas usando un heatmap.

In [97]:
# Correlation matrix
corr = df.corr()
sns.heatmap(corr, annot=True)
plt.show()

ValueError: could not convert string to float: '02 Albacete'

# 11. TESTs
## cleaning.py


Vamos a usar una función dada por ChatGPT que lo que hace es: "Python busca módulos un nivel más arriba"

In [101]:
import sys
import os

# Añadir la raíz del proyecto al path
project_root = os.path.abspath("..")
if project_root not in sys.path:
    sys.path.append(project_root)

In [103]:
from src.io import load_csv
from src.cleaning import clean
from src.config import RAW_PATH

df_raw = load_csv(RAW_PATH)
df_clean = clean(df_raw)

df_clean.head()
df_clean.shape

(3484, 9)